In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../../"))

In [2]:
import brmspy.rpy2_session as RS
import numpy as np

In [3]:
rs = RS.RSession(
    runtime_conf={
        # "r_home": "...",
        "startup": [
            # "install.packages('MASS', repos='https://cloud.r-project.org')",
        ],
    }
)

In [4]:
get_data = rs.fun("data")
get_data("epilepsy", package="brms")
df = rs.r("epilepsy")
df

,Age,Base,Trt,patient,visit,count,obs,zAge,zBase
1,31.0,11.0,1,1,1,5.0,1,0.424995,-0.757173
2,30.0,11.0,1,2,1,3.0,2,0.265284,-0.757173
3,25.0,6.0,1,3,1,2.0,3,-0.533274,-0.944403
4,36.0,8.0,1,4,1,4.0,4,1.223553,-0.869511
5,22.0,66.0,1,5,1,7.0,5,-1.012408,1.302363
...,...,...,...,...,...,...,...,...,...
232,32.0,16.0,2,55,4,3.0,232,0.584707,-0.569942
233,26.0,22.0,2,56,4,8.0,233,-0.373562,-0.345266
234,21.0,25.0,2,57,4,1.0,234,-1.172120,-0.232927
235,36.0,13.0,2,58,4,0.0,235,1.223553,-0.682281


In [5]:
brms = rs.package("brms")

In [6]:
formula = "count ~ zAge + zBase * Trt + (1|patient)"
family = "poisson"
result = brms.brm(
    formula=formula,
    data=df,
    family=family,
    warmup=500,
    iter=1000,
    chains=4
)


R callback write-console: Compiling Stan program...
  
R callback write-console: Start sampling
  



SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 7.8e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.78 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:   1 / 1000 [  0%]  (Warmup)
Chain 1: Iteration: 100 / 1000 [ 10%]  (Warmup)
Chain 1: Iteration: 200 / 1000 [ 20%]  (Warmup)
Chain 1: Iteration: 300 / 1000 [ 30%]  (Warmup)
Chain 1: Iteration: 400 / 1000 [ 40%]  (Warmup)
Chain 1: Iteration: 500 / 1000 [ 50%]  (Warmup)
Chain 1: Iteration: 501 / 1000 [ 50%]  (Sampling)
Chain 1: Iteration: 600 / 1000 [ 60%]  (Sampling)
Chain 1: Iteration: 700 / 1000 [ 70%]  (Sampling)
Chain 1: Iteration: 800 / 1000 [ 80%]  (Sampling)
Chain 1: Iteration: 900 / 1000 [ 90%]  (Sampling)
Chain 1: Iteration: 1000 / 1000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.116 seconds (Warm-up)
Chain 1:                0.709 seconds (Sampling)
Chain 1:                1.825 seconds (

RSessionError: 'NULLType' object is not iterable

Remote traceback:
Traceback (most recent call last):
  File "/Users/sebastian/PycharmProjects/pybrms/brmspy/rpy2_session/worker.py", line 90, in worker_main
    out = r_to_py(out_r)
          ^^^^^^^^^^^^^^
  File "/Users/sebastian/PycharmProjects/pybrms/brmspy/helpers/conversion.py", line 1092, in r_to_py
    result[key] = r_to_py(obj.rx2(name))
                  ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sebastian/PycharmProjects/pybrms/brmspy/helpers/conversion.py", line 1060, in r_to_py
    return cv.rpy2py(obj)
           ^^^^^^^^^^^^^^
  File "/Users/sebastian/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12/functools.py", line 912, in wrapper
    return dispatch(args[0].__class__)(*args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sebastian/PycharmProjects/pybrms/.venv/lib/python3.12/site-packages/rpy2/robjects/pandas2ri.py", line 336, in rpy2py_dataframe
    od = OrderedDict(
      

In [10]:
shape = (3000, 4000)
arr = np.arange(shape[0] * shape[1], dtype="float64").reshape(*shape)
ident = rs.fun("function(x) x")
out = ident(arr)
display(arr)
display(out)

array([[0.0000000e+00, 1.0000000e+00, 2.0000000e+00, ..., 3.9970000e+03,
        3.9980000e+03, 3.9990000e+03],
       [4.0000000e+03, 4.0010000e+03, 4.0020000e+03, ..., 7.9970000e+03,
        7.9980000e+03, 7.9990000e+03],
       [8.0000000e+03, 8.0010000e+03, 8.0020000e+03, ..., 1.1997000e+04,
        1.1998000e+04, 1.1999000e+04],
       ...,
       [1.1988000e+07, 1.1988001e+07, 1.1988002e+07, ..., 1.1991997e+07,
        1.1991998e+07, 1.1991999e+07],
       [1.1992000e+07, 1.1992001e+07, 1.1992002e+07, ..., 1.1995997e+07,
        1.1995998e+07, 1.1995999e+07],
       [1.1996000e+07, 1.1996001e+07, 1.1996002e+07, ..., 1.1999997e+07,
        1.1999998e+07, 1.1999999e+07]], shape=(3000, 4000))

array([[0.0000000e+00, 1.0000000e+00, 2.0000000e+00, ..., 3.9970000e+03,
        3.9980000e+03, 3.9990000e+03],
       [4.0000000e+03, 4.0010000e+03, 4.0020000e+03, ..., 7.9970000e+03,
        7.9980000e+03, 7.9990000e+03],
       [8.0000000e+03, 8.0010000e+03, 8.0020000e+03, ..., 1.1997000e+04,
        1.1998000e+04, 1.1999000e+04],
       ...,
       [1.1988000e+07, 1.1988001e+07, 1.1988002e+07, ..., 1.1991997e+07,
        1.1991998e+07, 1.1991999e+07],
       [1.1992000e+07, 1.1992001e+07, 1.1992002e+07, ..., 1.1995997e+07,
        1.1995998e+07, 1.1995999e+07],
       [1.1996000e+07, 1.1996001e+07, 1.1996002e+07, ..., 1.1999997e+07,
        1.1999998e+07, 1.1999999e+07]], shape=(3000, 4000))